In [1]:
import pandas as pd
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = 10000
import numpy as np
import logging
import multiprocessing
import time

In [2]:
col1 = [ 'Ha', 'pHa', 'La', 'LotNumA', 'lot_shape__A', 'lot_shape_symm_A', 'lot_shape_L_A', 'lot_shape_R_A',
             'Hb', 'pHb', 'Lb', 'LotNumB',  'lot_shape__B', 'lot_shape_symm_B', 'lot_shape_L_B', 'lot_shape_R_B',
             'Corr',
             'B1',
             'Beast1',  
             'diffEV', 'diffSDs', 'diffMins', 'diffMaxs', 'diffUV', 'RatioMin', 'SignMax',
             'Dom', 'diffBEV0', 'diffSignEV', 'pBbet_Unbiased1', 'pBbet_Uniform',  'pBbet_Sign1']


In [3]:
columns_20= []
for i in range(1,21): 
    columns_20.extend([str(str(co) + '_'+ str(i)) for co in col1])

x_col = columns_20
y_col = ['sigma', 'kapa', 'beta', 'gama']

In [26]:
df = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/synth_all_kapa.csv')

In [27]:
PF_games = pd.read_csv('/home/meghanmergui/Synth20/Create_Synthetic_Games/2_Add_PF/Synth10KPF.csv')
df = df.merge(PF_games, on=['Ha', 'pHa', 'La', 'LotShapeA', 'LotNumA', 'Hb', 'pHb', 'Lb', 'LotShapeB', 'LotNumB', 
                           'Corr', 'Amb'], how='left')


In [28]:
## Add lot shape columns 
def lot_shape_convert(lot_shape):
    if lot_shape == '-' : 
        return 1,0,0,0
    elif lot_shape == 'Symm' : 
        return 0,1,0,0
    elif lot_shape == 'L-skew' : 
        return 0,0,1,0
    elif lot_shape == 'R-skew' :
        return 0,0,0,1


In [29]:
df['lot_shape__B'], df['lot_shape_symm_B'], df['lot_shape_L_B'], df['lot_shape_R_B'] = zip(*df['LotShapeB'].map(lot_shape_convert))
df['lot_shape__A'], df['lot_shape_symm_A'], df['lot_shape_L_A'], df['lot_shape_R_A'] = zip(*df['LotShapeA'].map(lot_shape_convert))
    

In [47]:
# df1= df.copy()
df = df1.sample(1800)

In [48]:
columns_20= []
for i in range(1,21): 
    columns_20.extend([str(str(co) + '_'+ str(i)) for co in col1])

x_col = columns_20
y_col = ['sigma', 'kapa', 'beta', 'gama']
    

In [49]:
kapa = 1
df_indivK = df.loc[(df.kapa == kapa)]
sigma = 3.5 
beta = 1.3
gama = 0.25

In [50]:
num_subj = df.shape[0]/(20*3)

In [51]:
num_subj

30.0

In [52]:
df_indivK.shape

(600, 46)

In [53]:
df_indivK_subj = df_indivK.copy()
df_indivK_subj['SubjID'] = 'K'
df_indivK['SubjID']= 'K'
num_g = (kapa-1)*num_subj


/home/meghanmergui@st.technion.ac.il/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [54]:
df_all20 = pd.DataFrame(columns=x_col+y_col+['SubjID'])

In [55]:
df_indivK.columns

Index(['Ha', 'pHa', 'La', 'LotNumA', 'LotShapeA', 'Hb', 'pHb', 'Lb', 'LotNumB',
       'LotShapeB', 'Corr', 'Amb', 'GameId', 'B1', 'sigma', 'kapa', 'beta',
       'gama', 'psi', 'theta', 'wamb', 'Beast1', 'diffEV', 'diffSDs',
       'diffMins', 'diffMaxs', 'diffUV', 'RatioMin', 'SignMax',
       'pBbet_Unbiased1', 'pBbet_UnbiasedFB', 'pBbet_Uniform', 'pBbet_Sign1',
       'pBbet_SignFB', 'Dom', 'diffBEV0', 'diffBEVfb', 'diffSignEV',
       'lot_shape__B', 'lot_shape_symm_B', 'lot_shape_L_B', 'lot_shape_R_B',
       'lot_shape__A', 'lot_shape_symm_A', 'lot_shape_L_A', 'lot_shape_R_A',
       'SubjID'],
      dtype='object')

In [56]:
for g in range(df_indivK.shape[0]) : 
    if df_indivK.shape[0] >= 20:
        if df_indivK.shape[0]%20==0:
            print(df_indivK.shape)
            logging.info('{}: process : {}, shape : {}'.format((time.asctime(time.localtime(time.time()))), kapa, df_indivK.shape))
        df_20 = df_indivK.sample(20)
        print(df_20.GameId.unique())
        if df_20.GameId.nunique() == 20: 
            print('True')
            df_indivK_subj.at[df_20.index, 'SubjID'] = num_g
            df_indivK = df_indivK.drop(df_20.index)
            df_20 = df_20[col1]
            new_row = list(np.reshape(df_20.values, (1,df_20.shape[0]*df_20.shape[1]))[0])
            new_row.extend([sigma, kapa, beta, gama, num_g])
            df_all20.loc[df_all20.shape[0]] = new_row
            num_g +=1     

(600, 47)
[ 136 1951  468 5595 6200 1365  471 1981  565 3708  310 1040  303  664
 4994 2043 4233 4088 5443 7159]
True
(580, 47)
[2053 3297 4588 8052 2265 7163 9648 7941 7327 2654 2383 9421 8193 7348
 6286 4808 5950 4415 2317 9608]
True
(560, 47)
[6820 3117 5896  860  127 6307 1907 9379 4051 1088 5945 3433 2384 1612
 5006 5063  849 5270 1769 2677]
True
(540, 47)
[8538 9526 7110  310 8944 1735 4143  541 2404 3107 6883 6294 7579 6165
 4689 5589 7263 4654 4879 8637]
True
(520, 47)
[5297 7649 2095 6719 1279 5393 3297 9477 3298 8681 6441 4809 2150 2992
 7075 5087 9861 5965 6171  731]
True
(500, 47)
[5636 6469 2879 8612 3780 6195  375 2810 5382 3652 3424 7243 5209   22
 6638 1527 4274 9475  606 7492]
True
(480, 47)
[9689 8364 6705 9793 4738 8734 2350 6155   78 8851 5697 2064 2748 1383
 3091 2196 2101  194 5771 2437]
True
(460, 47)
[9170 3936 8123 6216 7006 4985 9791 3083 5089 7063 8993 1842 4416 1025
  787 3205 7151 2646 1540 2560]
True
(440, 47)
[5018 4510 7243 5602 9332 3851 6180 6450  232 

In [57]:
df_all20

,Ha_1,pHa_1,La_1,LotNumA_1,lot_shape__A_1,lot_shape_symm_A_1,lot_shape_L_A_1,lot_shape_R_A_1,Hb_1,pHb_1,Lb_1,LotNumB_1,lot_shape__B_1,lot_shape_symm_B_1,lot_shape_L_B_1,lot_shape_R_B_1,Corr_1,B1_1,Beast1_1,diffEV_1,diffSDs_1,diffMins_1,diffMaxs_1,diffUV_1,RatioMin_1,SignMax_1,Dom_1,diffBEV0_1,diffSignEV_1,pBbet_Unbiased1_1,pBbet_Uniform_1,pBbet_Sign1_1,Ha_2,pHa_2,La_2,LotNumA_2,lot_shape__A_2,lot_shape_symm_A_2,lot_shape_L_A_2,lot_shape_R_A_2,Hb_2,pHb_2,Lb_2,LotNumB_2,lot_shape__B_2,lot_shape_symm_B_2,lot_shape_L_B_2,lot_shape_R_B_2,Corr_2,B1_2,Beast1_2,diffEV_2,diffSDs_2,diffMins_2,diffMaxs_2,diffUV_2,RatioMin_2,SignMax_2,Dom_2,diffBEV0_2,diffSignEV_2,pBbet_Unbiased1_2,pBbet_Uniform_2,pBbet_Sign1_2,Ha_3,pHa_3,La_3,LotNumA_3,lot_shape__A_3,lot_shape_symm_A_3,lot_shape_L_A_3,lot_shape_R_A_3,Hb_3,pHb_3,Lb_3,LotNumB_3,lot_shape__B_3,lot_shape_symm_B_3,lot_shape_L_B_3,lot_shape_R_B_3,Corr_3,B1_3,Beast1_3,diffEV_3,diffSDs_3,diffMins_3,diffMaxs_3,diffUV_3,RatioMin_3,SignMax_3,Dom_3,diffBEV0_3,diffSignEV_3,pBbet_Unbiased1_3,pBbet_Uniform_3,pBbet_Sign1_3,Ha_4,pHa_4,La_4,LotNumA_4,lot_shape__A_4,lot_shape_symm_A_4,lot_shape_L_A_4,lot_shape_R_A_4,Hb_4,pHb_4,Lb_4,LotNumB_4,lot_shape__B_4,lot_shape_symm_B_4,lot_shape_L_B_4,lot_shape_R_B_4,Corr_4,B1_4,Beast1_4,diffEV_4,diffSDs_4,diffMins_4,diffMaxs_4,diffUV_4,RatioMin_4,SignMax_4,Dom_4,diffBEV0_4,diffSignEV_4,pBbet_Unbiased1_4,pBbet_Uniform_4,pBbet_Sign1_4,Ha_5,pHa_5,La_5,LotNumA_5,lot_shape__A_5,lot_shape_symm_A_5,lot_shape_L_A_5,lot_shape_R_A_5,Hb_5,pHb_5,Lb_5,LotNumB_5,lot_shape__B_5,lot_shape_symm_B_5,lot_shape_L_B_5,lot_shape_R_B_5,Corr_5,B1_5,Beast1_5,diffEV_5,diffSDs_5,diffMins_5,diffMaxs_5,diffUV_5,RatioMin_5,SignMax_5,Dom_5,diffBEV0_5,diffSignEV_5,pBbet_Unbiased1_5,pBbet_Uniform_5,pBbet_Sign1_5,Ha_6,pHa_6,La_6,LotNumA_6,lot_shape__A_6,lot_shape_symm_A_6,lot_shape_L_A_6,lot_shape_R_A_6,Hb_6,pHb_6,Lb_6,LotNumB_6,lot_shape__B_6,lot_shape_symm_B_6,lot_shape_L_B_6,lot_shape_R_B_6,Corr_6,B1_6,Beast1_6,diffEV_6,diffSDs_6,diffMins_6,diffMaxs_6,diffUV_6,RatioMin_6,SignMax_6,Dom_6,diffBEV0_6,diffSignEV_6,pBbet_Unbiased1_6,pBbet_Uniform_6,pBbet_Sign1_6,Ha_7,pHa_7,La_7,LotNumA_7,lot_shape__A_7,lot_shape_symm_A_7,lot_shape_L_A_7,lot_shape_R_A_7,Hb_7,pHb_7,Lb_7,LotNumB_7,lot_shape__B_7,lot_shape_symm_B_7,lot_shape_L_B_7,lot_shape_R_B_7,Corr_7,B1_7,Beast1_7,diffEV_7,diffSDs_7,diffMins_7,diffMaxs_7,diffUV_7,RatioMin_7,SignMax_7,Dom_7,diffBEV0_7,diffSignEV_7,pBbet_Unbiased1_7,pBbet_Uniform_7,pBbet_Sign1_7,Ha_8,pHa_8,La_8,LotNumA_8,lot_shape__A_8,lot_shape_symm_A_8,lot_shape_L_A_8,lot_shape_R_A_8,Hb_8,pHb_8,Lb_8,LotNumB_8,lot_shape__B_8,lot_shape_symm_B_8,lot_shape_L_B_8,lot_shape_R_B_8,Corr_8,B1_8,Beast1_8,diffEV_8,diffSDs_8,diffMins_8,diffMaxs_8,diffUV_8,RatioMin_8,SignMax_8,Dom_8,diffBEV0_8,diffSignEV_8,pBbet_Unbiased1_8,pBbet_Uniform_8,pBbet_Sign1_8,Ha_9,pHa_9,La_9,LotNumA_9,lot_shape__A_9,lot_shape_symm_A_9,lot_shape_L_A_9,lot_shape_R_A_9,Hb_9,pHb_9,Lb_9,LotNumB_9,lot_shape__B_9,lot_shape_symm_B_9,lot_shape_L_B_9,lot_shape_R_B_9,Corr_9,B1_9,Beast1_9,diffEV_9,diffSDs_9,diffMins_9,diffMaxs_9,diffUV_9,RatioMin_9,SignMax_9,Dom_9,diffBEV0_9,diffSignEV_9,pBbet_Unbiased1_9,pBbet_Uniform_9,pBbet_Sign1_9,Ha_10,pHa_10,La_10,LotNumA_10,lot_shape__A_10,lot_shape_symm_A_10,lot_shape_L_A_10,lot_shape_R_A_10,Hb_10,pHb_10,Lb_10,LotNumB_10,lot_shape__B_10,lot_shape_symm_B_10,lot_shape_L_B_10,lot_shape_R_B_10,Corr_10,B1_10,Beast1_10,diffEV_10,diffSDs_10,diffMins_10,diffMaxs_10,diffUV_10,RatioMin_10,SignMax_10,Dom_10,diffBEV0_10,diffSignEV_10,pBbet_Unbiased1_10,pBbet_Uniform_10,pBbet_Sign1_10,Ha_11,pHa_11,La_11,LotNumA_11,lot_shape__A_11,lot_shape_symm_A_11,lot_shape_L_A_11,lot_shape_R_A_11,Hb_11,pHb_11,Lb_11,LotNumB_11,lot_shape__B_11,lot_shape_symm_B_11,lot_shape_L_B_11,lot_shape_R_B_11,Corr_11,B1_11,Beast1_11,diffEV_11,diffSDs_11,diffMins_11,diffMaxs_11,diffUV_11,RatioMin_11,SignMax_11,Dom_11,diffBEV0_11,diffSignEV_11,pBbet_Unbiased1_11,pBbet_Uniform_11,pBbet_Sign1_11,Ha_12,pHa_12,La_12,LotNumA_12,lot_shape__

In [68]:
from random import sample 
subj_test = sample(list(df_all20.SubjID), k=int(df_all20.shape[0]*0.1))


In [69]:
df_all20_train = df_all20.loc[~(df_all20.SubjID.isin(subj_test))]
df_all20_test = df_all20.loc[(df_all20.SubjID.isin(subj_test))]
df_indivK_subj_train = df_indivK_subj.loc[~(df_indivK_subj.SubjID.isin(subj_test))]
df_indivK_subj_test = df_indivK_subj.loc[(df_indivK_subj.SubjID.isin(subj_test))]


In [73]:
df_all20_train

,Ha_1,pHa_1,La_1,LotNumA_1,lot_shape__A_1,lot_shape_symm_A_1,lot_shape_L_A_1,lot_shape_R_A_1,Hb_1,pHb_1,Lb_1,LotNumB_1,lot_shape__B_1,lot_shape_symm_B_1,lot_shape_L_B_1,lot_shape_R_B_1,Corr_1,B1_1,Beast1_1,diffEV_1,diffSDs_1,diffMins_1,diffMaxs_1,diffUV_1,RatioMin_1,SignMax_1,Dom_1,diffBEV0_1,diffSignEV_1,pBbet_Unbiased1_1,pBbet_Uniform_1,pBbet_Sign1_1,Ha_2,pHa_2,La_2,LotNumA_2,lot_shape__A_2,lot_shape_symm_A_2,lot_shape_L_A_2,lot_shape_R_A_2,Hb_2,pHb_2,Lb_2,LotNumB_2,lot_shape__B_2,lot_shape_symm_B_2,lot_shape_L_B_2,lot_shape_R_B_2,Corr_2,B1_2,Beast1_2,diffEV_2,diffSDs_2,diffMins_2,diffMaxs_2,diffUV_2,RatioMin_2,SignMax_2,Dom_2,diffBEV0_2,diffSignEV_2,pBbet_Unbiased1_2,pBbet_Uniform_2,pBbet_Sign1_2,Ha_3,pHa_3,La_3,LotNumA_3,lot_shape__A_3,lot_shape_symm_A_3,lot_shape_L_A_3,lot_shape_R_A_3,Hb_3,pHb_3,Lb_3,LotNumB_3,lot_shape__B_3,lot_shape_symm_B_3,lot_shape_L_B_3,lot_shape_R_B_3,Corr_3,B1_3,Beast1_3,diffEV_3,diffSDs_3,diffMins_3,diffMaxs_3,diffUV_3,RatioMin_3,SignMax_3,Dom_3,diffBEV0_3,diffSignEV_3,pBbet_Unbiased1_3,pBbet_Uniform_3,pBbet_Sign1_3,Ha_4,pHa_4,La_4,LotNumA_4,lot_shape__A_4,lot_shape_symm_A_4,lot_shape_L_A_4,lot_shape_R_A_4,Hb_4,pHb_4,Lb_4,LotNumB_4,lot_shape__B_4,lot_shape_symm_B_4,lot_shape_L_B_4,lot_shape_R_B_4,Corr_4,B1_4,Beast1_4,diffEV_4,diffSDs_4,diffMins_4,diffMaxs_4,diffUV_4,RatioMin_4,SignMax_4,Dom_4,diffBEV0_4,diffSignEV_4,pBbet_Unbiased1_4,pBbet_Uniform_4,pBbet_Sign1_4,Ha_5,pHa_5,La_5,LotNumA_5,lot_shape__A_5,lot_shape_symm_A_5,lot_shape_L_A_5,lot_shape_R_A_5,Hb_5,pHb_5,Lb_5,LotNumB_5,lot_shape__B_5,lot_shape_symm_B_5,lot_shape_L_B_5,lot_shape_R_B_5,Corr_5,B1_5,Beast1_5,diffEV_5,diffSDs_5,diffMins_5,diffMaxs_5,diffUV_5,RatioMin_5,SignMax_5,Dom_5,diffBEV0_5,diffSignEV_5,pBbet_Unbiased1_5,pBbet_Uniform_5,pBbet_Sign1_5,Ha_6,pHa_6,La_6,LotNumA_6,lot_shape__A_6,lot_shape_symm_A_6,lot_shape_L_A_6,lot_shape_R_A_6,Hb_6,pHb_6,Lb_6,LotNumB_6,lot_shape__B_6,lot_shape_symm_B_6,lot_shape_L_B_6,lot_shape_R_B_6,Corr_6,B1_6,Beast1_6,diffEV_6,diffSDs_6,diffMins_6,diffMaxs_6,diffUV_6,RatioMin_6,SignMax_6,Dom_6,diffBEV0_6,diffSignEV_6,pBbet_Unbiased1_6,pBbet_Uniform_6,pBbet_Sign1_6,Ha_7,pHa_7,La_7,LotNumA_7,lot_shape__A_7,lot_shape_symm_A_7,lot_shape_L_A_7,lot_shape_R_A_7,Hb_7,pHb_7,Lb_7,LotNumB_7,lot_shape__B_7,lot_shape_symm_B_7,lot_shape_L_B_7,lot_shape_R_B_7,Corr_7,B1_7,Beast1_7,diffEV_7,diffSDs_7,diffMins_7,diffMaxs_7,diffUV_7,RatioMin_7,SignMax_7,Dom_7,diffBEV0_7,diffSignEV_7,pBbet_Unbiased1_7,pBbet_Uniform_7,pBbet_Sign1_7,Ha_8,pHa_8,La_8,LotNumA_8,lot_shape__A_8,lot_shape_symm_A_8,lot_shape_L_A_8,lot_shape_R_A_8,Hb_8,pHb_8,Lb_8,LotNumB_8,lot_shape__B_8,lot_shape_symm_B_8,lot_shape_L_B_8,lot_shape_R_B_8,Corr_8,B1_8,Beast1_8,diffEV_8,diffSDs_8,diffMins_8,diffMaxs_8,diffUV_8,RatioMin_8,SignMax_8,Dom_8,diffBEV0_8,diffSignEV_8,pBbet_Unbiased1_8,pBbet_Uniform_8,pBbet_Sign1_8,Ha_9,pHa_9,La_9,LotNumA_9,lot_shape__A_9,lot_shape_symm_A_9,lot_shape_L_A_9,lot_shape_R_A_9,Hb_9,pHb_9,Lb_9,LotNumB_9,lot_shape__B_9,lot_shape_symm_B_9,lot_shape_L_B_9,lot_shape_R_B_9,Corr_9,B1_9,Beast1_9,diffEV_9,diffSDs_9,diffMins_9,diffMaxs_9,diffUV_9,RatioMin_9,SignMax_9,Dom_9,diffBEV0_9,diffSignEV_9,pBbet_Unbiased1_9,pBbet_Uniform_9,pBbet_Sign1_9,Ha_10,pHa_10,La_10,LotNumA_10,lot_shape__A_10,lot_shape_symm_A_10,lot_shape_L_A_10,lot_shape_R_A_10,Hb_10,pHb_10,Lb_10,LotNumB_10,lot_shape__B_10,lot_shape_symm_B_10,lot_shape_L_B_10,lot_shape_R_B_10,Corr_10,B1_10,Beast1_10,diffEV_10,diffSDs_10,diffMins_10,diffMaxs_10,diffUV_10,RatioMin_10,SignMax_10,Dom_10,diffBEV0_10,diffSignEV_10,pBbet_Unbiased1_10,pBbet_Uniform_10,pBbet_Sign1_10,Ha_11,pHa_11,La_11,LotNumA_11,lot_shape__A_11,lot_shape_symm_A_11,lot_shape_L_A_11,lot_shape_R_A_11,Hb_11,pHb_11,Lb_11,LotNumB_11,lot_shape__B_11,lot_shape_symm_B_11,lot_shape_L_B_11,lot_shape_R_B_11,Corr_11,B1_11,Beast1_11,diffEV_11,diffSDs_11,diffMins_11,diffMaxs_11,diffUV_11,RatioMin_11,SignMax_11,Dom_11,diffBEV0_11,diffSignEV_11,pBbet_Unbiased1_11,pBbet_Uniform_11,pBbet_Sign1_11,Ha_12,pHa_12,La_12,LotNumA_12,lot_shape__

In [58]:
print(list(df_all20.SubjID))

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0]


In [59]:
pd.DataFrame(df_indivK_subj.SubjID.value_counts()).reset_index().sort_values('SubjID')

,index,SubjID
10,10,20
18,17,20
17,16,20
16,15,20
15,14,20
14,13,20
13,12,20
12,11,20
11,19,20
19,18,20


In [ ]:
df_all20.shape()

## Check Testing the code

In [28]:
df_train20 = pd.read_csv('/home/meghanmergui/Synth20/SynthData/Train20_1_test.csv')

In [29]:
df_train20

,Ha_1,pHa_1,La_1,LotNumA_1,lot_shape__A_1,lot_shape_symm_A_1,lot_shape_L_A_1,lot_shape_R_A_1,Hb_1,pHb_1,Lb_1,LotNumB_1,lot_shape__B_1,lot_shape_symm_B_1,lot_shape_L_B_1,lot_shape_R_B_1,Corr_1,B1_1,Beast1_1,diffEV_1,diffSDs_1,diffMins_1,diffMaxs_1,diffUV_1,RatioMin_1,SignMax_1,Dom_1,diffBEV0_1,diffSignEV_1,pBbet_Unbiased1_1,pBbet_Uniform_1,pBbet_Sign1_1,Ha_2,pHa_2,La_2,LotNumA_2,lot_shape__A_2,lot_shape_symm_A_2,lot_shape_L_A_2,lot_shape_R_A_2,Hb_2,pHb_2,Lb_2,LotNumB_2,lot_shape__B_2,lot_shape_symm_B_2,lot_shape_L_B_2,lot_shape_R_B_2,Corr_2,B1_2,Beast1_2,diffEV_2,diffSDs_2,diffMins_2,diffMaxs_2,diffUV_2,RatioMin_2,SignMax_2,Dom_2,diffBEV0_2,diffSignEV_2,pBbet_Unbiased1_2,pBbet_Uniform_2,pBbet_Sign1_2,Ha_3,pHa_3,La_3,LotNumA_3,lot_shape__A_3,lot_shape_symm_A_3,lot_shape_L_A_3,lot_shape_R_A_3,Hb_3,pHb_3,Lb_3,LotNumB_3,lot_shape__B_3,lot_shape_symm_B_3,lot_shape_L_B_3,lot_shape_R_B_3,Corr_3,B1_3,Beast1_3,diffEV_3,diffSDs_3,diffMins_3,diffMaxs_3,diffUV_3,RatioMin_3,SignMax_3,Dom_3,diffBEV0_3,diffSignEV_3,pBbet_Unbiased1_3,pBbet_Uniform_3,pBbet_Sign1_3,Ha_4,pHa_4,La_4,LotNumA_4,lot_shape__A_4,lot_shape_symm_A_4,lot_shape_L_A_4,lot_shape_R_A_4,Hb_4,pHb_4,Lb_4,LotNumB_4,lot_shape__B_4,lot_shape_symm_B_4,lot_shape_L_B_4,lot_shape_R_B_4,Corr_4,B1_4,Beast1_4,diffEV_4,diffSDs_4,diffMins_4,diffMaxs_4,diffUV_4,RatioMin_4,SignMax_4,Dom_4,diffBEV0_4,diffSignEV_4,pBbet_Unbiased1_4,pBbet_Uniform_4,pBbet_Sign1_4,Ha_5,pHa_5,La_5,LotNumA_5,lot_shape__A_5,lot_shape_symm_A_5,lot_shape_L_A_5,lot_shape_R_A_5,Hb_5,pHb_5,Lb_5,LotNumB_5,lot_shape__B_5,lot_shape_symm_B_5,lot_shape_L_B_5,lot_shape_R_B_5,Corr_5,B1_5,Beast1_5,diffEV_5,diffSDs_5,diffMins_5,diffMaxs_5,diffUV_5,RatioMin_5,SignMax_5,Dom_5,diffBEV0_5,diffSignEV_5,pBbet_Unbiased1_5,pBbet_Uniform_5,pBbet_Sign1_5,Ha_6,pHa_6,La_6,LotNumA_6,lot_shape__A_6,lot_shape_symm_A_6,lot_shape_L_A_6,lot_shape_R_A_6,Hb_6,pHb_6,Lb_6,LotNumB_6,lot_shape__B_6,lot_shape_symm_B_6,lot_shape_L_B_6,lot_shape_R_B_6,Corr_6,B1_6,Beast1_6,diffEV_6,diffSDs_6,diffMins_6,diffMaxs_6,diffUV_6,RatioMin_6,SignMax_6,Dom_6,diffBEV0_6,diffSignEV_6,pBbet_Unbiased1_6,pBbet_Uniform_6,pBbet_Sign1_6,Ha_7,pHa_7,La_7,LotNumA_7,lot_shape__A_7,lot_shape_symm_A_7,lot_shape_L_A_7,lot_shape_R_A_7,Hb_7,pHb_7,Lb_7,LotNumB_7,lot_shape__B_7,lot_shape_symm_B_7,lot_shape_L_B_7,lot_shape_R_B_7,Corr_7,B1_7,Beast1_7,diffEV_7,diffSDs_7,diffMins_7,diffMaxs_7,diffUV_7,RatioMin_7,SignMax_7,Dom_7,diffBEV0_7,diffSignEV_7,pBbet_Unbiased1_7,pBbet_Uniform_7,pBbet_Sign1_7,Ha_8,pHa_8,La_8,LotNumA_8,lot_shape__A_8,lot_shape_symm_A_8,lot_shape_L_A_8,lot_shape_R_A_8,Hb_8,pHb_8,Lb_8,LotNumB_8,lot_shape__B_8,lot_shape_symm_B_8,lot_shape_L_B_8,lot_shape_R_B_8,Corr_8,B1_8,Beast1_8,diffEV_8,diffSDs_8,diffMins_8,diffMaxs_8,diffUV_8,RatioMin_8,SignMax_8,Dom_8,diffBEV0_8,diffSignEV_8,pBbet_Unbiased1_8,pBbet_Uniform_8,pBbet_Sign1_8,Ha_9,pHa_9,La_9,LotNumA_9,lot_shape__A_9,lot_shape_symm_A_9,lot_shape_L_A_9,lot_shape_R_A_9,Hb_9,pHb_9,Lb_9,LotNumB_9,lot_shape__B_9,lot_shape_symm_B_9,lot_shape_L_B_9,lot_shape_R_B_9,Corr_9,B1_9,Beast1_9,diffEV_9,diffSDs_9,diffMins_9,diffMaxs_9,diffUV_9,RatioMin_9,SignMax_9,Dom_9,diffBEV0_9,diffSignEV_9,pBbet_Unbiased1_9,pBbet_Uniform_9,pBbet_Sign1_9,Ha_10,pHa_10,La_10,LotNumA_10,lot_shape__A_10,lot_shape_symm_A_10,lot_shape_L_A_10,lot_shape_R_A_10,Hb_10,pHb_10,Lb_10,LotNumB_10,lot_shape__B_10,lot_shape_symm_B_10,lot_shape_L_B_10,lot_shape_R_B_10,Corr_10,B1_10,Beast1_10,diffEV_10,diffSDs_10,diffMins_10,diffMaxs_10,diffUV_10,RatioMin_10,SignMax_10,Dom_10,diffBEV0_10,diffSignEV_10,pBbet_Unbiased1_10,pBbet_Uniform_10,pBbet_Sign1_10,Ha_11,pHa_11,La_11,LotNumA_11,lot_shape__A_11,lot_shape_symm_A_11,lot_shape_L_A_11,lot_shape_R_A_11,Hb_11,pHb_11,Lb_11,LotNumB_11,lot_shape__B_11,lot_shape_symm_B_11,lot_shape_L_B_11,lot_shape_R_B_11,Corr_11,B1_11,Beast1_11,diffEV_11,diffSDs_11,diffMins_11,diffMaxs_11,diffUV_11,RatioMin_11,SignMax_11,Dom_11,diffBEV0_11,diffSignEV_11,pBbet_Unbiased1_11,pBbet_Uniform_11,pBbet_Sign1_11,Ha_12,pHa_12,La_12,LotNumA_12,lot_shape__

In [30]:
df_train = pd.read_csv('/home/meghanmergui/Synth20/SynthData/Train_1_test.csv')

In [31]:
df_train

,Ha,pHa,La,LotNumA,LotShapeA,Hb,pHb,Lb,LotNumB,LotShapeB,Corr,Amb,GameId,B1,sigma,kapa,beta,gama,psi,theta,wamb,Beast1,diffEV,diffSDs,diffMins,diffMaxs,diffUV,RatioMin,SignMax,pBbet_Unbiased1,pBbet_UnbiasedFB,pBbet_Uniform,pBbet_Sign1,pBbet_SignFB,Dom,diffBEV0,diffBEVfb,diffSignEV,lot_shape__B,lot_shape_symm_B,lot_shape_L_B,lot_shape_R_B,lot_shape__A,lot_shape_symm_A,lot_shape_L_A,lot_shape_R_A,SubjID
0,53,0.25,-20,1,-,86,0.25,-23,1,-,0,0,1,1.0,3.5,1,1.3,0.25,0.024479,0.970278,0,0.927667,6.000000e+00,15.588457,-3.0,33.0,15.000000,0.869565,1.0,-0.4962,-0.5010,0.0014,0.0000,-0.0040,0,6.000000e+00,6.000000e+00,0.000000e+00,1,0,0,0,1,0,0,0,4
1,42,0.01,9,4,R-skew,9,1.00,9,1,-,0,0,2,0.0,3.5,1,1.3,0.25,0.020403,0.952783,0,0.000000,-3.300000e-01,-3.315283,0.0,-44.0,-28.400000,1.000000,1.0,-0.0110,-0.0082,-0.7909,0.0000,0.0000,-1,-3.300000e-01,-3.300000e-01,7.105427e-15,1,0,0,0,0,0,0,1,7
2,-4,1.00,-4,1,-,5,0.99,-20,4,Symm,0,0,3,1.0,3.5,1,1.3,0.25,0.048057,0.321642,0,0.890667,8.750000e+00,2.632489,-16.0,10.5,4.000000,0.200000,1.0,0.9804,0.9838,0.5854,0.9910,0.9885,0,8.750000e+00,8.750000e+00,5.247000e+01,0,1,0,0,1,0,0,0,7
3,13,1.00,13,1,-,54,0.40,-9,1,-,0,0,4,0.6,3.5,1,1.3,0.25,0.035201,0.901778,0,0.379333,3.200000e+00,30.863571,-22.0,41.0,9.500000,0.000000,1.0,-0.1992,-0.1986,0.0094,-0.5980,-0.6006,0,3.200000e+00,3.200000e+00,-7.560000e+01,1,0,0,0,1,0,0,0,6
4,-6,1.00,-6,1,-,-6,1.00,-6,8,L-skew,0,0,5,0.6,3.5,1,1.3,0.25,0.014610,0.301809,0,0.495333,0.000000e+00,26.172505,-247.0,7.0,-54.750000,0.023715,1.0,0.7474,0.7520,-0.2656,0.5044,0.5076,0,0.000000e+00,0.000000e+00,2.540000e+02,0,0,1,0,1,0,0,0,5
5,36,0.25,15,1,-,30,0.90,-19,1,-,0,0,6,0.2,3.5,1,1.3,0.25,0.062864,0.304762,0,0.425000,4.850000e+00,5.606733,-34.0,-6.0,-20.000000,0.000000,1.0,0.3020,0.3430,-1.0000,-0.1019,-0.0999,0,4.850000e+00,4.850000e+00,-1.100000e+01,1,0,0,0,1,0,0,0,1
6,30,1.00,30,2,L-skew,30,1.00,30,1,-,0,0,7,0.4,3.5,1,1.3,0.25,0.025285,0.583492,0,0.505333,0.000000e+00,-1.000000,1.0,-1.0,0.000000,0.966667,1.0,-0.0134,-0.0186,0.0032,0.0000,0.0000,0,0.000000e+00,0.000000e+00,0.000000e+00,1,0,0,0,0,0,1,0,3
7,38,0.80,-16,4,L-skew,35,0.50,24,1,-,0,0,8,0.2,3.5,1,1.3,0.25,0.060278,0.370680,0,0.688667,2.300000e+00,-16.485450,40.0,-6.0,4.300000,0.000000,1.0,-0.5960,-0.4501,-0.5009,0.1995,0.1961,0,2.300000e+00,2.300000e+00,2.280000e+01,1,0,0,0,0,0,1,0,3
8,-1,1.00,-1,1,-,75,0.20,-20,6,L-skew,0,0,9,0.6,3.5,1,1.3,0.25,0.004538,0.115158,0,0.496667,1.110223e-16,38.369259,-19.0,81.0,50.428571,0.050000,1.0,-0.6012,-0.5894,0.7116,0.2049,0.2001,0,1.110223e-16,1.110223e-16,4.000000e+01,0,0,1,0,1,0,0,0,5
9,23,1.00,23,2,Symm,23,1.00,23,1,-,0,0,10,0.8,3.5,1,1.3,0.25,0.014665,0.843082,0,0.514333,0.000000e+00,-0.500000,0.5,-0.5,0.000000,0.978261,1.0,-0.0152,-0.0168,-0.0060,0.0000,0.0000,0,0.000000e+00,0.000000e+00,0.000000e+00,1,0,0,0,0,1,0,0,6


In [32]:
df_train.SubjID.value_counts()

9    20
8    20
7    20
6    20
5    20
4    20
3    20
2    20
1    20
0    20
Name: SubjID, dtype: int64

In [33]:
df_train.SubjID.unique()

array([4, 7, 6, 5, 1, 3, 8, 0, 2, 9])

In [35]:
df_test20.SubjID.value_counts()

9.0    1
8.0    1
7.0    1
6.0    1
5.0    1
4.0    1
3.0    1
2.0    1
1.0    1
0.0    1
Name: SubjID, dtype: int64

In [37]:
df_test20.SubjID.unique()

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

# Rearrange Train and Test 

In [1]:
import pandas as pd

In [58]:
kapa = 3

In [59]:
# Save DataFrame by 20 games per row 
df_all20_train = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Train20_' + str(kapa) +str('.csv')))

In [60]:
df_all20_train.SubjID.nunique()

99900

In [61]:
subj2700 = df_all20_train.sample(900)

In [62]:
df_all20_train = df_all20_train.drop(subj2700.index)

In [63]:
df_all20_train.to_csv(str('/home/meghanmergui/Synth20/SynthData/ppTrain20_' + str(kapa) +str('.csv')), index = False)

In [64]:
df_all20_test = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Test20_' + str(kapa) +str('.csv')))

In [65]:
df_all20_test.columns

Index(['Ha_1', 'pHa_1', 'La_1', 'LotNumA_1', 'lot_shape__A_1',
       'lot_shape_symm_A_1', 'lot_shape_L_A_1', 'lot_shape_R_A_1', 'Hb_1',
       'pHb_1',
       ...
       'diffBEV0_20', 'diffSignEV_20', 'pBbet_Unbiased1_20',
       'pBbet_Uniform_20', 'pBbet_Sign1_20', 'sigma', 'kapa', 'beta', 'gama',
       'SubjID'],
      dtype='object', length=645)

In [66]:
df_all20_test = pd.concat([df_all20_test, subj2700], axis=0)

In [67]:
df_all20_test.to_csv(str('/home/meghanmergui/Synth20/SynthData/ppTest20_' + str(kapa) +str('.csv')), index = False)

In [68]:
df_indivK_subj_train = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Train_' + str(kapa) +str('.csv')))

In [69]:
df_indivK_subj_train.shape

(1998000, 47)

In [70]:
indivk2700 = df_indivK_subj_train.loc[df_indivK_subj_train.SubjID.isin(subj2700.SubjID)]

In [71]:
indivk2700.shape

(18000, 47)

In [72]:
df_indivK_subj_train = df_indivK_subj_train.drop(indivk2700.index)

In [73]:
df_indivK_subj_train.shape

(1980000, 47)

In [74]:
df_indivK_subj_train.to_csv(str('/home/meghanmergui/Synth20/SynthData/ppTrain_' + str(kapa) +str('.csv')), index = False)

In [75]:
df_indivK_subj_test = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Test_' + str(kapa) +str('.csv')))


In [76]:
df_indivK_subj_test = pd.concat([df_indivK_subj_test, indivk2700], axis = 0)

In [77]:
df_indivK_subj_test.shape

(20000, 47)

In [78]:
df_indivK_subj_train.shape

(1980000, 47)

In [79]:
df_indivK_subj_test.to_csv(str('/home/meghanmergui/Synth20/SynthData/ppTest_' + str(kapa) +str('.csv')), index = False)


In [3]:

# Save DataFrame with SujbID, one game per row, train and test separatly : 
import pandas as pd

kapa = 1
df_indivK_subj_train = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Train_' + str(kapa) +str('.csv')))
df_indivK_subj_test = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Test_' + str(kapa) +str('.csv')))

# Save DataFrame by 20 games per row 
df_all20_train = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Train20_' + str(kapa) +str('.csv')))
df_all20_test = pd.read_csv(str('/home/meghanmergui/Synth20/SynthData/Test20_' + str(kapa) +str('.csv')))
    

KeyboardInterrupt: 

In [ ]:
df_all20_train.Subj